# Test #5 - The combined influence of topography, water elevation level and bedrock elevation

## Initialization

In [1]:
# Load pyKasso and other useful packages
import numpy as np
import pykasso as pk
app = pk.pykasso()

In [2]:
# Create new project and set project settings
project_path = 'geometry_05'
grid_parameters = {
    'x0' : 0,
    'y0' : 0,
    'z0' : 100,
    'nx' : 100,
    'ny' : 100,
    'nz' : 50,
    'dx' : 10,
    'dy' : 10,
    'dz' : 10
}
app.new_project(grid_parameters=grid_parameters, project_name=project_path)

In [3]:
# Construct the model parameters dictionary
model_parameters = {
    'sks' : {
        'seed' : 5555,
    },
    'domain' : {},
    'outlets' : {
        'number'     : 1,
        'data'       : [[994, 500]],
        'importance' : [1],
        'mode'       : 'domain_bottom',
    },
    'inlets' : {
        'number'     : 2,
        'data'       : [[0, 250], [0, 994]],
        'per_outlet' : [1],
        'importance' : [1],
        'mode'       : 'domain_surface',
    },
}

In [4]:
X, Y, Z = app.project.grid.get_meshgrids()
x = X[:,:,0]
y = Y[:,:,0]

# Construct the topography
topography = np.sin(y/120)*80 + 500

# Construct the water level elevation
water_level_elevation = np.linspace(400, 300, 100)
water_level_elevation = np.repeat(water_level_elevation[:, np.newaxis], 100, axis=1)
water_level_elevation = np.sin(x/40)*7 + water_level_elevation

# Construct the bedrock
z_valley = np.linspace(300, 100, 100)
bedrock = []
for z in z_valley:
    line = np.append(np.linspace(200+z, z, 50), np.linspace(z, 200+z, 50)) 
    bedrock.append(line)
bedrock = np.vstack(bedrock)

# Visualize the arrays
# app.visualizer.show_array(topography)
# app.visualizer.show_array(water_level_elevation)
# app.visualizer.show_array(bedrock)

# Update the parameters
model_parameters['domain']['topography']  = topography
model_parameters['domain']['water_level'] = water_level_elevation
model_parameters['domain']['bedrock']     = bedrock

In [5]:
# Compute the karstic networks
for algorithm in ['Isotropic3', 'Riemann3']:
    model_parameters['sks']['algorithm'] = algorithm
    app.model.generate(model_parameters)

A


In [6]:
# Visualize the results
settings = {
    'ghost_values': [0],
    'outline' : True,
    'inlets'  : True,
    'outlets' : True,
    'surfaces' : {'topography':1, 'water_level':1, 'bedrock': 1},
}
app.visualizer.pv_show([0,1], ['karst'], settings=settings)